# 第3章 字典和集合

> dict是实现python的基石，dict的底层实现基于哈希表

Python 的基础数据类型中的字典类型分为：无序字典 与 有序字典 两种类型

无序字典: `dict()`

有序字典：
```
import collections

my_order_dict = collections.OrderedDict()
```

## 字典的现代句法


In [2]:
dial_codes = [
    (880, 'Bangladesh'),
    (55, 'Brazil'),
    (86, 'China'),
    (91, 'India'),
    (62, 'Indonesia'),
    (81, 'Japan'),
    (234, 'Nigeria'),
    (92, 'Pakistan'),
    (7, 'Russia'),
    (1, 'United States'),
]
# 对调键和值
country_dial = {country: code for code, country in dial_codes}
country_dial


{'Bangladesh': 880,
 'Brazil': 55,
 'China': 86,
 'India': 91,
 'Indonesia': 62,
 'Japan': 81,
 'Nigeria': 234,
 'Pakistan': 92,
 'Russia': 7,
 'United States': 1}

In [4]:
{code: country.upper() for country,code in sorted(country_dial.items())  if code >70 } #再次对调排序

{880: 'BANGLADESH',
 86: 'CHINA',
 91: 'INDIA',
 81: 'JAPAN',
 234: 'NIGERIA',
 92: 'PAKISTAN'}

In [10]:
{code: country.upper() for code,country in sorted(dial_codes,key = lambda i:(i[1],i[0]))  if code >70 } # 同样的效果 

{880: 'BANGLADESH',
 86: 'CHINA',
 91: 'INDIA',
 81: 'JAPAN',
 234: 'NIGERIA',
 92: 'PAKISTAN'}

### 映射拆包

In [11]:
def dump(**kwargs):  ## 调用函数时，不止一个参数可以使用**
    return kwargs

dump(**{'x':1},y=2,**{'z':3})

{'x': 1, 'y': 2, 'z': 3}

In [12]:
# ** 在 dict 字面量中使用，同样可以多次使用

{'a':0,**{'x':1},'y':2,**{'z':3,'x':4}}

{'a': 0, 'x': 4, 'y': 2, 'z': 3}

 ### 使用  | 合并映射  

 要求：python3.9

In [15]:
d1 = {'a':1,'b':3}
d2 = {'a':2,'b':4,'c':6}

d1 | d2  # 合并 

{'a': 2, 'b': 4, 'c': 6}

In [18]:
dict(d1,**d2)  # 等同于

{'a': 2, 'b': 4, 'c': 6}

In [20]:
dict(list(d1.items()) + list(d2.items())) # 等同于

{'a': 2, 'b': 4, 'c': 6}

In [21]:
{**d1,**d2} # 等同于

{'a': 2, 'b': 4, 'c': 6}

### 使用模式匹配处理映射

In [22]:
# 从出版物记录中提取创作者的名字
def get_creators(record: dict) -> list:
    match record:
        case {'type': 'book', 'api': 2, 'authors': [*names]}:  # 'authors' 键映 射一个序列的映射对象,返回列表
            return names
        case {'type': 'book', 'api': 1, 'author': name}: #'authors' 键映 射任何对象的映射对象，返回列表
            return [name]
        case {'type': 'book'}:
            return ValueError(f"Invalid 'book' record: {record!r}")
        case {'type': 'movie', 'director': name}:
            return [name]
        case _:
            return ValueError(f'Invalid record: {record!r}')

In [24]:
b1 = dict(api=1, author='Douglas Hofstadter', type='book', title='Godel, Escher, Bach')
get_creators(b1)

['Douglas Hofstadter,ok']

In [36]:
#Python split() 通过指定分隔符对字符串进行切片,默认空格
b2 = dict(api=2, authors='Martelli,Ravenscroft,Holden'.split(','), type='book', title='Godel, Escher, Bach')
get_creators(b2)

['Martelli', 'Ravenscroft', 'Holden']

In [37]:
from collections import OrderedDict

b2 = OrderedDict(api=2, type='book', title='Python in a Nutshell', authors='Martelli Ravenscroft Holden'.split())
get_creators(b2)

['Martelli', 'Ravenscroft', 'Holden']

In [38]:
get_creators({'type': 'book', 'pages': 770})

ValueError("Invalid 'book' record: {'type': 'book', 'pages': 770}")

In [39]:
get_creators('Spam, spam, spam')

ValueError("Invalid record: 'Spam, spam, spam'")

In [40]:
food = dict(category = 'ice cream',flavor='vanilla',cost=199)

match food:
    case {'category':'ice cream', **details}:
        print(f'Ice cream details: {details}')

Ice cream details: {'flavor': 'vanilla', 'cost': 199}


### 映射类型的标准API

In [46]:
from collections import abc
my_dict = {}
isinstance(my_dict,abc.Mapping)

True

In [47]:
isinstance(my_dict, abc.MutableMapping)

True

In [50]:
tt = (1,2,(30,40))
hash(tt)

-3907003130834322577

In [53]:
tl = (1, 2, [30, 40])
hash(tl)

TypeError: unhashable type: 'list'

In [54]:
tf = (1, 2, frozenset([30, 40])) # frozenset() 返回一个冻结的集合
hash(tf)

5149391500123939311

In [55]:
b = frozenset('runoob')  # 创建不可变集合
b

frozenset({'b', 'n', 'o', 'r', 'u'})

### 自动处理丢失的键

人为设置的值主要有两种方法：

- 第一种是把普通的 dict 换成 defaultdict；

- 第二种是定义 dict 或其他映射类型的子 类，实现 __missing__ 方法

In [58]:
# 在查找键时把非字符串键转换成字符串
class StrKeyDict0(dict):
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError:
            return default
        
    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys()


In [59]:
d = StrKeyDict0([('2', 'two'), ('4', 'four')])
d['2']

'two'

In [60]:
d[4]

'four'

In [61]:
d.get('2')

'two'

In [63]:
d.get('4')

'four'

In [64]:
d.get(1, 'N/A')

'N/A'

In [77]:
2 in d

True

In [78]:
1 in d

False

### dict的变体

1.collections.OrderedDict  

等值检查考虑  顺序
方便执行重新排序操作，空间利用率、迭代速度和更新操作的性能是次     要的。
从算法上看，OrderedDict处理频繁重新排序操作的效果比dict好，适合用于跟踪近期存         
串键与Python对象之间的映射。


情况。
2.collection     inMap

存放一组映射，可作为一  个整体来搜索。
查找操作按照输入映射在构造函数调用中出现的顺序执行，一旦找到指定  的键，立即结束。
不复制输入映射  ，存放映射的引用。
更新或插入操作只    
影键与Python对象之间的映射。

In [81]:
d1 = dict(a=1, b=3)
d2 = dict(a=2, b=4, c=6)

In [82]:
from collections import ChainMap
chain = ChainMap(d1, d2)
chain

ChainMap({'a': 1, 'b': 3}, {'a': 2, 'b': 4, 'c': 6})

In [84]:
chain['a']

1

In [85]:
chain['c']

6

In [86]:
chain['c'] = -1
d1

{'a': 1, 'b': 3, 'c': -1}

In [87]:
d2

{'a': 2, 'b': 4, 'c': 6}

In [88]:
import builtins 
pylookup = ChainMap(locals(), globals(), vars(builtins)

SyntaxError: incomplete input (1906800348.py, line 2)

3.collections.Counter

一种对键计数的映射，更新现有的键，计数随之增加。

可用于统计可哈希对象的实例数量。


In [89]:
# 使用 Counter 统计词 中的字母数量。
ct = collections.Counter('abracadabra')
ct

Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})

In [91]:
# 更新后再统计
ct.update('aaaaazzz')
ct

Counter({'a': 15, 'z': 6, 'b': 2, 'r': 2, 'c': 1, 'd': 1})

In [92]:
ct.most_common(3)  # 注意，'b' 和 'r' 两个键并列第三，但是 ct.most_common(3) 只 显示 3 项。

[('a', 15), ('z', 6), ('b', 2)]


4.shelve.Shelf

持久存储字符串键与Python对象之间的映射。

### 子类应继承 UserDict 而不是 dict

In [ ]:
# StrKeyDict在插入、更新和查找时，始终把非字符串键转换为str类型
import collections

class StrKeyDict(collections.UserDict):
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def __contains__(self, key):
        return str(key) in self.data
    
    def __setitem__(self, key, item):
        self.data[str(key)] = item


## 不可变映射

In [2]:
from types import  MappingProxyType
d = {1:'A'}
d_proxy = MappingProxyType(d)
d_proxy[1]

'A'

In [3]:
d_proxy[1] ='B'   # 不可修改，也不可新增
d_proxy[1]

TypeError: 'mappingproxy' object does not support item assignment

In [4]:
d[2] = 'B'  
d_proxy

mappingproxy({1: 'A', 2: 'B'})

In [5]:
d_proxy[2]

'B'

### 字典试图

In [7]:
d = dict(a =10,b=20,c=30)
values = d.values()
print(values)
print(d)
print(len(values))
print(list(values))
values[0]   #不能使用 [] 获取视图中的项


dict_values([10, 20, 30])
{'a': 10, 'b': 20, 'c': 30}
3
[10, 20, 30]


TypeError: 'dict_values' object is not subscriptable

In [9]:
d['z'] = 99
print(d)
print(values)

{'a': 10, 'b': 20, 'c': 30, 'z': 99}
dict_values([10, 20, 30, 99])


In [8]:
val = [10, 20, 30]
val[0]

10

### 集合论

集合指 set  和  frozenset ,集合是一组唯一的对象，自动去重

In [10]:
l = ['spam', 'spam', 'eggs', 'spam', 'bacon', 'eggs']
set(l)

{'bacon', 'eggs', 'spam'}

In [11]:
list(set(l))

['spam', 'bacon', 'eggs']

In [17]:
dict.fromkeys(l).keys()  

d = (dict.fromkeys(l).keys()) # 保留每一项首次出现的位置的顺序

In [18]:
l

['spam', 'spam', 'eggs', 'spam', 'bacon', 'eggs']

In [21]:
len(d & l)  # 比较两个集合中出现共同的次数

3

In [22]:
frozenset(range(10))

frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9})

### 集合推导式


In [24]:
from unicodedata import name

{chr(i) for i in range(32, 256) if 'SIGN' in name(chr(i),'') }  # chr() 用一个范围在 range（256）内的（就是0～255）整数作参数，返回一个对应的字符。

{'#',
 '$',
 '%',
 '+',
 '<',
 '=',
 '>',
 '¢',
 '£',
 '¤',
 '¥',
 '§',
 '©',
 '¬',
 '®',
 '°',
 '±',
 'µ',
 '¶',
 '×',
 '÷'}

In [25]:
d1 = dict(a=1, b=2, c=3, d=4)
d2 = dict(b=20, d=40, e=50)
d1.keys() & d2.keys()

{'b', 'd'}

In [26]:
s = {'a', 'e', 'i'}
d1.keys() & s

{'a'}

In [27]:
d1.keys() | s

{'a', 'b', 'c', 'd', 'e', 'i'}

In [56]:
# http://localhost/datawhale/Relph1119/fluent-python/docs/#/contents/ch03